In [ ]:
import os
from glob import glob

import pairwise_fte
from py_utils import data_ops
import evaluate_fte
from lib import app
from py_utils import log

%load_ext autoreload
%autoreload 2

%matplotlib inline

root_dir = os.path.join("..", "data")
# root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")\
logger = log.logger("main")

In [ ]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = pairwise_fte.create_pose_functions(root_dir)

In [ ]:
# Configurations for the optimisation. 2017_08_29/top/phantom/run1_1 2017_08_29/top/phantom/flick1_1 2019_02_27/kiara/run
data_path = os.path.join("2019_03_09", "lily", "run")
start_frame = 80
end_frame = 180
dlc_thresh = 0.5

In [ ]:
# Run the optimisation
pairwise_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, init_ekf=True, loss="lsq", generate_reprojection_videos=True, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
pairwise_fte.plot_cheetah(root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
pairwise_fte.compare_cheetahs("/Users/zico/msc/dev/AcinoSet/data/2019_03_09/lily/run/fte_pw_1/fte.pickle", root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# DEBUB purposes - if you want to display particular frames to inspect the measurements that are incorporated in the optimisation process.
# The camera to use.
cam_num = 1
# The measurements to display 1 - base prediction, 2 - 3 pairwise measurements. Max is 3.
pw_values = (1, 2, 3)
# Frame number to display.
frame_num = 110
pairwise_fte.display_test_image(data_dir, cam_num, pw_values, frame_num)

In [ ]:
eval_dir = os.path.join("/Users/zico/msc/dev/AcinoSet/data", data_path, "fte_pw")
data = data_ops.load_pickle(os.path.join(eval_dir, "fte.pickle"))
results_dir = os.path.join(eval_dir, "evaluation")
os.makedirs(eval_dir, exist_ok=True)
max_delta_acc = evaluate_fte.eval_delta_acc(data, results_dir)
evaluate_fte.eval_model_error(data, results_dir)
evaluate_fte.eval_meas_error(data, results_dir)
Q = [ # model parameters variance
        max_delta_acc[0], max_delta_acc[1], max_delta_acc[2], # x, y, z
        max_delta_acc[3], max_delta_acc[6], 0, max_delta_acc[10], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, #  phi_1, ... , phi_14
        max_delta_acc[4], max_delta_acc[7], max_delta_acc[9], max_delta_acc[11], max_delta_acc[13], max_delta_acc[15], max_delta_acc[17], max_delta_acc[18], max_delta_acc[19], max_delta_acc[20], max_delta_acc[21], max_delta_acc[22], max_delta_acc[23], max_delta_acc[24], # theta_1, ... , theta_n
        max_delta_acc[5], max_delta_acc[8], 0, max_delta_acc[12], max_delta_acc[14], max_delta_acc[16], 0, 0, 0, 0, 0, 0, 0, 0, # psi_1, ... , psi_n
    #     ?, ?, ? # lure's x, y, z variance
    ]
logger.info(Q)

In [ ]:
fig_fpath = os.path.join(os.path.join(os.path.join("..", "data"), data_path, "fte_pw"), 'fte.svg')
pw_file = data_ops.load_pickle(os.path.join(os.path.join("..", "data"), data_path, "fte_pw", "fte.pickle"))
app.plot_cheetah_states(pw_file["x"], out_fpath=fig_fpath)

In [ ]:
pairwise_fte.plot_cost_functions()